# Data Arrangement

## Load Packages

In [4]:
library(foreign)
library(tidyr)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Load Data
The sav files belong to the project owner and are not prepared by me. Please ignore the undecalred levels in these sav files.

In [7]:
#Participant Log
log = read.spss('../../Data/DemosDx/Tx Day Consent Log_1.sav', to.data.frame = T)

#Depression Measure
cudos = read.spss('../../Data/DailyMeasures/CUDOS_1.sav', reencode = NA, to.data.frame = T)

#Anxiety Measure
cuxos = read.spss('../../Data/DailyMeasures/CUXOS_1.sav', reencode = NA, to.data.frame = T)

#Anger Measure
cuangos = read.spss('../../Data/DailyMeasures/CUANGOS_1.sav', reencode = NA, to.data.frame = T)

#ACT Component Measure
compact = read.spss('../../Data/DailyMeasures/COMPACT_1.sav', reencode = NA, to.data.frame = T)

#Acceptance Commitment Measure
act = read.spss('../../Data/DailyMeasures/ACT_1.sav', reencode = NA, to.data.frame = T)

#Demographic
demo = read.spss('../../Data/DemosDx/Demographics Form_1.sav', reencode = NA, to.data.frame = T)

#Preprocess 
preprocess = function(x){
    x = as.data.frame(lapply(x, as.numeric)) 
    x = x[,-grep('F', colnames(x))]
    x = x[unique(x$ID1),]
    return(x)
}

cudos = preprocess(cudos)
cuxos = preprocess(cuxos)
cuangos = preprocess(cuangos)
compact = as.data.frame(lapply(compact, as.numeric))
compact = compact[unique(compact$ID1),]
act = preprocess(act)


Warning message in read.spss("../../Data/DemosDx/Tx Day Consent Log_1.sav", to.data.frame = T):
“Undeclared level(s) 0, 11, 12 added in variable: DC_status_1”
Warning message in read.spss("../../Data/DemosDx/Tx Day Consent Log_1.sav", to.data.frame = T):
“Duplicated levels in factor DC_status_1: Other”
Warning message in read.spss("../../Data/DemosDx/Tx Day Consent Log_1.sav", to.data.frame = T):
“Undeclared level(s) 2 added in variable: SCID_1”
Warning message in read.spss("../../Data/DemosDx/Tx Day Consent Log_1.sav", to.data.frame = T):
“Undeclared level(s) 0 added in variable: Referral_1”
Warning message in read.spss("../../Data/DailyMeasures/CUDOS_1.sav", reencode = NA, :
“Undeclared level(s) 11 added in variable: dep6_14_1”
Warning message in read.spss("../../Data/DailyMeasures/CUDOS_1.sav", reencode = NA, :
“Undeclared level(s) 9 added in variable: dep8_1_1”
Warning message in read.spss("../../Data/DailyMeasures/CUDOS_1.sav", reencode = NA, :
“Undeclared level(s) 5 added in vari

Warning message in read.spss("../../Data/DailyMeasures/CUANGOS_1.sav", reencode = NA, :
“Undeclared level(s) 32 added in variable: Ang2_1_1”
Warning message in read.spss("../../Data/DailyMeasures/CUANGOS_1.sav", reencode = NA, :
“Undeclared level(s) 10 added in variable: Ang2_6_1”
Warning message in read.spss("../../Data/DailyMeasures/CUANGOS_1.sav", reencode = NA, :
“Undeclared level(s) 10 added in variable: Ang3_2_1”
Warning message in read.spss("../../Data/DemosDx/Demographics Form_1.sav", reencode = NA, :
“Undeclared level(s) 6 added in variable: Race_1”
Warning message in read.spss("../../Data/DemosDx/Demographics Form_1.sav", reencode = NA, :
“Undeclared level(s) 8 added in variable: Relationship_1”


The ID in these files indicate a unique <i> admission </i>. However, one person can have multiple admissions, and we do not have any column indicating a unique patient ID. There is a column in the log file specifying the previous admission ID of each current admission (NA for first admission). We can create unique patient IDs based on this info. However, for hospital data, I highly recommend the team to log patient ID instead of tracking previous admission ID manually.


In [9]:
#Create unique patient ID
log$uniquePtId = NA

for (i in 1:nrow(log)){
    if (is.na(log$Prev_AdmitID_1[i])){
        log$uniquePtId[i] = log$ID1[i]    
    }else{
        prev = log$Prev_AdmitID_1[i]
        while(!is.na(log$Prev_AdmitID_1[which(log$ID1 == prev)])){
            prev = log$Prev_AdmitID_1[which(log$ID1 == prev)]
        }
        log$uniquePtId[i] = prev
    }

    
}

In [10]:
#Merge different data sets by admission ID 
dailyLog = merge(cudos, log[,c('ID1','SCID_1','Days_complete_1','DC_status_1')], by = 'ID1', all.x = T)
dailyLog = merge(dailyLog, cuxos, by = 'ID1', all.x = T)
dailyLog = merge(dailyLog, cuangos, by = 'ID1', all.x = T)
dailyLog = merge(dailyLog, compact, by = 'ID1', all.x = T)
dailyLog = merge(dailyLog, act, by.x = 'ID1',by.y = 'ID', all.x = T)


## Examine basic info

In [15]:
#Treatment Completion Table
table(dailyLog$DC_status_1) #Treatment Completion: N = 3792

#Treatment Completion Table
table(dailyLog$DC_status_1)/nrow(dailyLog)*100 #Treatment Complete: 63.16%


                                                      0 
                                                      1 
                                     Treatment Complete 
                                                   3792 
                              Treatment nearly complete 
                                                    195 
                           Insurance no longer covering 
                                                     86 
                                          Nonattendance 
                                                    790 
Inappropriate (language)/Referred to another tx setting 
                                                    100 
                               Transferred to inpatient 
                                                    204 
                   Withdrew due to external limitations 
                                                    361 
                 Reported dissatisfaction, withdrew AMA 
                              


                                                      0 
                                             0.01665556 
                                     Treatment Complete 
                                            63.15789474 
                              Treatment nearly complete 
                                             3.24783478 
                           Insurance no longer covering 
                                             1.43237841 
                                          Nonattendance 
                                            13.15789474 
Inappropriate (language)/Referred to another tx setting 
                                             1.66555630 
                               Transferred to inpatient 
                                             3.39773484 
                   Withdrew due to external limitations 
                                             6.01265823 
                 Reported dissatisfaction, withdrew AMA 
                              

In [16]:
#Mean Treatment Lenghth for Patients who completed treatment 
meanCompDay = mean(dailyLog$Days_complete_1, na.rm = T) 
print(paste('Completion Day Mean:', round(meanCompDay, 2)))
print(paste('Completion Day SD:', round(sd(dailyLog$Days_complete_1, na.rm = T), 2)))
print(paste('Completion Day Range:', min(dailyLog$Days_complete_1, na.rm = T), 
            max(dailyLog$Days_complete_1, na.rm = T)))

[1] "Completion Day Mean: 8.12"
[1] "Completion Day SD: 5.39"
[1] "Completion Day Range: 0 78"


In [19]:
#Examine SCID'd sample 
table(dailyLog$SCID_1)


  no  yes    2 
3797 1993  214 

### Data Cleaning

According to the research team, there were some errorneous data entries with unidentifibal causes. For example, SCID'd patients should have data entry at time 0, while non SCID'd patient should not have data entry at time 0. They requested to exclude patients with such errors. 

In [26]:
#Exclude people who were not SCID'd but had information in their first data entry
dailyLogClean = dailyLog[-which((
                             rowSums(is.na(dailyLog[,grep('dep0', colnames(dailyLog))])) < 18 | 
                             rowSums(is.na(dailyLog[,grep('Ang0', colnames(dailyLog))])) < 14 |
                             rowSums(is.na(dailyLog[,grep('anx0', colnames(dailyLog))])) < 25) &
                             dailyLog$SCID_1 == 'no'),]

#Exclude people who were SCID'd but did not have information in their first data entry
dailyLogClean = dailyLogClean[-which((
                             rowSums(is.na(dailyLogClean[,grep('dep0', colnames(dailyLogClean))])) == 18 | 
                             rowSums(is.na(dailyLogClean[,grep('Ang0', colnames(dailyLogClean))])) == 14 |
                             rowSums(is.na(dailyLogClean[,grep('anx0', colnames(dailyLogClean))])) == 25) &
                             dailyLogClean$SCID_1 == 'yes'),]


In [27]:
length(unique(dailyLog$ID)) #6003

[1] 6003

In [28]:
length(unique(dailyLogClean$ID)) #3596

[1] 3596

## Reshape Data from wide to long format

In [32]:
#Depression
dfLong = 
        dailyLogClean[,grep(paste0('^ID|dep[0-9]*_1_'), colnames(dailyLogClean))]%>% 
        gather('day', 'dep1',-1)
dfLong$day = substring(dfLong$day, 4,5)
dfLong$day = gsub("_", "", dfLong$day)

for(i in 2:18){
    select = 
        dailyLogClean[,grep(paste0('dep[0-9]*_',i,'_'), colnames(dailyLogClean))]%>% 
        gather('day', 'dep')
    dfLong = cbind(dfLong, select$dep)
    colnames(dfLong)[i+2] = paste0('dep',i)
}

dfLong

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,dep9,dep10,dep11,dep12,dep13,dep14,dep15,dep16,dep17,dep18
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,0,5,5,5,1,4,1,5,2,5,5,5,5,5,5,4,4,5,5
5,0,5,5,1,5,5,1,1,5,5,5,5,5,5,3,2,5,5,5
6,0,3,3,4,1,1,2,3,3,5,3,5,3,3,1,3,4,4,3
9,0,1,2,4,1,4,1,5,3,3,1,1,4,1,1,1,1,1,2
13,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [33]:
#Anxiety
for(i in 1:25){
    select = 
        dailyLogClean[,grep(paste0('anx[0-9]*_',i,'_'), colnames(dailyLogClean))]%>% 
        gather('day', 'anx')
    select = rbind(select, data.frame(day = paste0('anx59_',i,'_1'), anx = rep(NA, nrow(dailyLogClean))))
    dfLong = cbind(dfLong, select$anx)
    colnames(dfLong)[i+20] = paste0('anx',i)
}
dfLong

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,anx16,anx17,anx18,anx19,anx20,anx21,anx22,anx23,anx24,anx25
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,0,5,5,5,1,4,1,5,2,⋯,3,1,1,3,1,NA,NA,NA,NA,NA
5,0,5,5,1,5,5,1,1,5,⋯,5,4,1,2,5,NA,NA,NA,NA,NA
6,0,3,3,4,1,1,2,3,3,⋯,1,1,1,2,1,NA,NA,NA,NA,NA
9,0,1,2,4,1,4,1,5,3,⋯,4,5,4,3,4,NA,NA,NA,NA,NA
13,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [34]:
#Anger
for(i in 1:14){
    select = 
        dailyLogClean[,grep(paste0('Ang[0-9]*_',i,'_'), colnames(dailyLogClean))]%>% 
        gather('day', 'ang')
    dfLong = cbind(dfLong, select$ang)
    colnames(dfLong)[i+45] = paste0('ang',i)
}
dfLong

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,ang5,ang6,ang7,ang8,ang9,ang10,ang11,ang12,ang13,ang14
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,0,5,5,5,1,4,1,5,2,⋯,1,1,1,1,1,1,1,1,1,1
5,0,5,5,1,5,5,1,1,5,⋯,2,1,3,3,3,1,1,1,3,1
6,0,3,3,4,1,1,2,3,3,⋯,5,5,4,5,4,3,3,3,2,2
9,0,1,2,4,1,4,1,5,3,⋯,3,2,1,2,2,1,1,1,2,1
13,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [35]:
#Act Components
for(i in 1:23){
    select = 
        dailyLogClean[,grep(paste0('comp[0-9]*_',i,'_'), colnames(dailyLogClean))]%>% 
        gather('day', 'comp')
    select = rbind(select, data.frame(day = paste0('comp59_',i,'_1'), comp = rep(NA, nrow(dailyLogClean))))
    dfLong = cbind(dfLong, select$comp)
    colnames(dfLong)[i+59] = paste0('comp',i)
}
dfLong

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,comp14,comp15,comp16,comp17,comp18,comp19,comp20,comp21,comp22,comp23
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,0,5,5,5,1,4,1,5,2,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,0,5,5,1,5,5,1,1,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,0,3,3,4,1,1,2,3,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,0,1,2,4,1,4,1,5,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
13,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [36]:
#ACT 
for(i in 1:19){
    select = 
        dailyLogClean[,grep(paste0('act[0-9]*_',i,'_'), colnames(dailyLogClean))]%>% 
        gather('day', 'act')
    select = rbind(select, data.frame(day = paste0('act59_',i,'_1'), act = rep(NA, nrow(dailyLogClean))))
    dfLong = cbind(dfLong, select$act)
    colnames(dfLong)[i+82] = paste0('act',i)
}
dfLong

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,act10,act11,act12,act13,act14,act15,act16,act17,act18,act19
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,0,5,5,5,1,4,1,5,2,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,0,5,5,1,5,5,1,1,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,0,3,3,4,1,1,2,3,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,0,1,2,4,1,4,1,5,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
13,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [37]:
#Recode non-scid'd patients' first session (their first data entry = session 0)
dfLong$day = as.numeric(dfLong$day)
dfLongAdj = dfLong
for (i in log$ID1[log$SCID_1!='yes']){
    dfLongAdj$day[which(dfLongAdj$ID1 == i)] = dfLongAdj$day[which(dfLongAdj$ID1 == i)]-1
}

dfLongAdj = dfLongAdj[-which(dfLongAdj$day == -1),]

In [39]:
#Remove rows without any data entires
dfLongAdj = dfLongAdj[which(rowSums(is.na(dfLongAdj))<99),]

In [40]:
dfLong

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,act10,act11,act12,act13,act14,act15,act16,act17,act18,act19
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,0,5,5,5,1,4,1,5,2,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,0,5,5,1,5,5,1,1,5,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,0,3,3,4,1,1,2,3,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,0,1,2,4,1,4,1,5,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
13,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
14,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,0,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [42]:
#Compute data completion rate
length(unique(dfLong$ID))
length(unique(dfLongAdj$ID)) #N = 3554
length(unique(dfLongAdj$ID))/length(unique(dfLong$ID)) #98.83% Data Completion Rate

[1] 3596

[1] 3554

[1] 0.9883204

In [44]:
#Examine treatment length
dfLongAdj %>%
group_by(ID1) %>%
filter(day == max(day)) %>%
ungroup()%>%
summarize(txLenM = mean(day), txLenSD = sd(day))  

txLenM,txLenSD
<dbl>,<dbl>
9.874262,6.226606


In [45]:
#Add summary variales 
dfLongAdj$depMean = rowMeans(dfLongAdj[,grep('dep[0-9]$|dep[1][0-6]',colnames(dfLongAdj))], na.rm = T)
dfLongAdj$anxMean = rowMeans(dfLongAdj[,grep('anx',colnames(dfLongAdj))], na.rm = T)
dfLongAdj$angMean = rowMeans(dfLongAdj[,grep('ang',colnames(dfLongAdj))], na.rm = T)
dfLongAdj = merge(dfLongAdj, log, by = 'ID1', x.all = T)
dfLongAdj = merge(dfLongAdj, demo, by = 'ID1', a.all = T)

In [46]:
write.csv(dfLongAdj, '../../Data/daily_dfLong.csv', row.names = F)

In [47]:
dfLongAdj

ID1,day,dep1,dep2,dep3,dep4,dep5,dep6,dep7,dep8,⋯,DOB_1.y,Sex_1,Gender_1,OtherGender_1,Race_1,Relationship_1,Education_1,Sexuality_1,OtherSexuality_1,PrimaryLast
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,4,1,1,1,1,4,1,1,1,⋯,11541744000,Female,Female,,White,Divorced,Graduated 2 year college,NA,,Primary Case
1,1,1,1,1,1,1,1,1,1,⋯,11541744000,Female,Female,,White,Divorced,Graduated 2 year college,NA,,Primary Case
1,2,1,1,2,1,4,1,1,1,⋯,11541744000,Female,Female,,White,Divorced,Graduated 2 year college,NA,,Primary Case
1,5,1,1,1,1,1,1,1,1,⋯,11541744000,Female,Female,,White,Divorced,Graduated 2 year college,NA,,Primary Case
1,0,1,1,2,2,5,1,3,1,⋯,11541744000,Female,Female,,White,Divorced,Graduated 2 year college,NA,,Primary Case
1,3,1,1,1,1,1,1,1,1,⋯,11541744000,Female,Female,,White,Divorced,Graduated 2 year college,NA,,Primary Case
1,6,1,1,1,1,1,1,1,1,⋯,11541744000,Female,Female,,White,Divorced,Graduated 2 year college,NA,,Primary Case
2,4,2,1,1,1,2,1,2,1,⋯,11474697600,Female,Female,,White,Widowed,Graduated 2 year college,NA,,Primary Case
2,6,1,2,1,5,5,2,2,2,⋯,11474697600,Female,Female,,White,Widowed,Graduated 2 year college,NA,,Primary Case


In [48]:
#Mean/SD for depression, anxiety, and anger mean scores at session 0

dfLongAdj %>%
filter(day == 0)%>%
summarize(depM = mean(depMean, na.rm = T), depSD = mean(depMean, na.rm = T),
         anxM = mean(anxMean, na.rm = T), anxSD = mean(anxMean, na.rm = T),
         angM = mean(angMean, na.rm = T), angSD = mean(angMean, na.rm = T))

depM,depSD,anxM,anxSD,angM,angSD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3.293032,3.293032,3.189613,3.189613,2.240602,2.240602


In [49]:
#Mean/SD for depression, anxiety, and anger mean scores at the last session

dfLongAdj %>%
group_by(ID1) %>%
filter(day == max(day))%>%
ungroup() %>%
summarize(depM = mean(depMean, na.rm = T), depSD = mean(depMean, na.rm = T),
         anxM = mean(anxMean, na.rm = T), anxSD = mean(anxMean, na.rm = T),
         angM = mean(angMean, na.rm = T), angSD = mean(angMean, na.rm = T))

depM,depSD,anxM,anxSD,angM,angSD
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2.38687,2.38687,2.413176,2.413176,1.601871,1.601871
